In [3]:
from taxi import utils, taxi

In [4]:
import gc
import glob
import torch


import numpy as np
from datasets import load_dataset
from graspologic.embed import ClassicalMDS
from scipy.spatial.distance import pdist, squareform
from sentence_transformers import SentenceTransformer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel,
    PeftConfig
)

EMBEDDING_MODEL = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True, device='cuda:0')

model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)
tokenizer.padding_token = tokenizer.eos_token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side='right'


# group 1: Health (2), Sports (5), Family (8)
# group 2: Politics (9), business (6), society and culture (0)
# group 3: Computers (4), Entertainment (7), Education (3) 
coarse_to_fine_dict = {
    0: (2,5,8),
    1: (9,6,0),
    2: (4,7,3)
}

dataset_name = 'community-datasets/yahoo_answers_topics'
topic_key = 'topic'
cache_dir = '~/.cache/huggingface/datasets'

N = 64
mc = 1

def get_fine_pis(coarse_pi, coarse_to_fine_dict):
    fine_topics = []
    fine_pi = []
    
    for i, coarse in enumerate(coarse_to_fine_dict):
        for fine in coarse_to_fine_dict[coarse]:
            fine_topics.append(fine)
            fine_pi.append(coarse_pi[coarse] / len(coarse_to_fine_dict[coarse]))
            
    return tuple(fine_topics), tuple(fine_pi)


<All keys matched successfully>


In [5]:
true_pi_list = [(1,0,0), (0,1,0), (0,0,1), (1/3,1/3,1/3)]
datasets = {}

for true_pi in true_pi_list:
    fine_topics, fine_pi = get_fine_pis(true_pi, coarse_to_fine_dict)
    datasets[true_pi] = utils.get_dataset_by_pi(fine_pi, dataset_name, topic_key, fine_topics, N, cache_dir=cache_dir, seed=mc)

In [6]:
dist_pis = squareform(pdist(true_pi_list))
dist_pis /= np.linalg.norm(dist_pis)
true_geometry = ClassicalMDS(n_components=3).fit_transform(dist_pis)

data_geometry = taxi.get_dataset_geometry(true_geometry, 
                                          datasets, 
                                          EMBEDDING_MODEL
                                         )

In [7]:
query_pi_list = [
    (1/3, 1/3, 1/3),
    (0.5, 0.25, 0.25),
    (0.75, 0.125, 0.125),
    (1, 0, 0)
]

query_datasets = {}

for true_pi in query_pi_list:
    fine_topics, fine_pi = get_fine_pis(true_pi, coarse_to_fine_dict)
    query_datasets[true_pi] = utils.get_dataset_by_pi(fine_pi, dataset_name, topic_key, fine_topics, N, cache_dir=cache_dir, seed=mc)

In [14]:
query_datasets[(1, 0, 0)][0]

{'id': 1315784,
 'topic': 5,
 'question_title': 'who won the third place play off in usa 1994?',
 'question_content': 'And what was the final score',
 'best_answer': 'Sweden beat Bulgaria 4-0 in the match for 3rd place at the Rose Bowl in Los Angeles in the 1994 FIFA World Cup held in USA.  The goals were scored by Thomas Brolin, Hakan Mild, Henrick Larsson, and Kennet Andersson.  Bulgaria, though had a slightly better record of the two during the first round (2 wins/1 loss vs 1 win/2 draws).'}

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
lora_matrices = {}
for save_string in glob.glob('./models/N_10000*lora_128*'):
    print(save_string)

    # Extract true_pi from save_string
    pi_str = save_string.split('pi_')[-1]
    pi_values = [int(x)/100 for x in pi_str.split('_')]
    true_pi = tuple(pi_values)
    print(f"Extracted true_pi: {true_pi}")

    base_model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            device_map="cuda:0",
            trust_remote_code=True
    )


    model = PeftModel.from_pretrained(
        base_model,
        save_string,
        torch_dtype=torch.float16,
        device_map='cuda:0'
    )

    loras = taxi.get_lora_matrices(model, target_modules = ["q_proj","k_proj","v_proj"])
    lora_matrices[true_pi] = loras

    #clear memory
    del model
    del base_model
    torch.cuda.empty_cache()
    gc.collect()

structural_geometry = taxi.get_structural_geometry(true_geometry, lora_matrices)

./models/N_10000_lora_128_id_0_pi_000_000_100
Extracted true_pi: (0.0, 0.0, 1.0)
./models/N_10000_lora_128_id_0_pi_000_100_000
Extracted true_pi: (0.0, 1.0, 0.0)


/home/ubuntu/helivan-project-generation/venvs/helivan/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


./models/N_10000_lora_128_id_0_pi_033_033_033
Extracted true_pi: (0.33, 0.33, 0.33)
./models/N_10000_lora_128_id_0_pi_100_000_000
Extracted true_pi: (1.0, 0.0, 0.0)


In [ ]:
from collections import defaultdict
func_behav = defaultdict(dict)
for save_string in glob.glob('./models/N_10000*lora_128*'):
    print('Model: ' , save_string)

    # Extract true_pi from save_string
    pi_str = save_string.split('pi_')[-1]
    pi_values = [int(x)/100 for x in pi_str.split('_')]
    true_pi = tuple(pi_values)

    base_model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            device_map="cuda:0",
            trust_remote_code=True
    )

    model = PeftModel.from_pretrained(
        base_model,
        save_string,
        torch_dtype=torch.float16,
        device_map='cuda:0'
    )

    for query_pi, query_dataset in query_datasets.items():
        print('\tQuerying with: ', query_pi)
        query_strings = [q['question_title'] for q in query_dataset]
        responses, hidden_states = taxi.get_outputs(model, tokenizer, query_strings, max_length=32, match_n_input_tokens=True)
        func_behav[true_pi][query_pi] = {'responses': responses, 'hidden_states': hidden_states}

    del model
    del base_model
    torch.cuda.empty_cache()
    gc.collect()

    

Model:  ./models/N_10000_lora_128_id_0_pi_000_000_100


/home/ubuntu/helivan-project-generation/venvs/helivan/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


	Querying with:  (0.3333333333333333, 0.3333333333333333, 0.3333333333333333)


/home/ubuntu/helivan-project-generation/venvs/helivan/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/ubuntu/helivan-project-generation/venvs/helivan/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ubuntu/helivan-project-generation/venvs/helivan/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


	Querying with:  (0.5, 0.25, 0.25)
	Querying with:  (0.75, 0.125, 0.125)
	Querying with:  (1, 0, 0)
Model:  ./models/N_10000_lora_128_id_0_pi_000_100_000
	Querying with:  (0.3333333333333333, 0.3333333333333333, 0.3333333333333333)
	Querying with:  (0.5, 0.25, 0.25)
	Querying with:  (0.75, 0.125, 0.125)
	Querying with:  (1, 0, 0)
Model:  ./models/N_10000_lora_128_id_0_pi_033_033_033
	Querying with:  (0.3333333333333333, 0.3333333333333333, 0.3333333333333333)
	Querying with:  (0.5, 0.25, 0.25)
	Querying with:  (0.75, 0.125, 0.125)
	Querying with:  (1, 0, 0)
Model:  ./models/N_10000_lora_128_id_0_pi_100_000_000
	Querying with:  (0.3333333333333333, 0.3333333333333333, 0.3333333333333333)
	Querying with:  (0.5, 0.25, 0.25)
	Querying with:  (0.75, 0.125, 0.125)
	Querying with:  (1, 0, 0)
